In [8]:
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications.resnet import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

In [ ]:
def load_data(path, width, height, batch_size):
    # cuando crees tu generador de datos, agrega reescalamiento, así como volteo horizontal y vertical.
        # revisa la documentación de Keras para más técnicas
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        horizontal_flip=True,
        vertical_flip=True
    )

    train_generator = train_datagen.flow_from_directory(
                                                        path,
                                                        target_size=(width, height),
                                                        batch_size=batch_size,
                                                        class_mode='categorical',
                                                        seed=12345,
                                                      )
    return train_generator

In [ ]:
def create_resnet_model(input_shape):
    backbone = ResNet50(
        include_top=False,  # No incluimos la capa de clasificación final
        weights='imagenet',  # Cargamos los pesos preentrenados en ImageNet
        input_shape=input_shape
    )
    # usaremos un modelo previamente entrenado con sus parámetros aprendidos como funciones para entrenar nuestro nuevo modelo
    for layer in backbone.layers:
        layer.trainable = False

    # Construyamos nuestro nuevo modelo capa por capa. Comienza utilizando la capa secuencial; después
    # agrega el backbone, una capa que aplana nuestra matriz en vectores y, por último, el
    # encabezado de la clasificación.  No olvides ajustar el número de clases que tu modelo
    # debe manejar

    model = Sequential()
    model.add(backbone)# aquí va tu modelo de preentrenamiento
    model.add(GlobalAveragePooling2D()) # aquí va la capa que aplana tu modelo de salida de preentrenamiento
    model.add(Dense(15, activation='softmax'))# aquí va tu encabezado de clasificación

    ## Define un optimizador de tu elección. No olvides pasar un
    # parámetro learning_rate adecuado.

    optimizer = Adam(learning_rate=0.001)  # Ajusta el learning rate según sea necesario

    ## compila tu modelo
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    return model

In [29]:
def train_model(model, train_data, val_data, epochs=10):
    model.fit(train_data, validation_data=val_data, epochs=epochs, verbose=2)
    return model

In [ ]:
## Parámetros
# Dependiendo de tu máquina, puedes jugar con los parámetros de ancho y alto
# para un entrenamiento más rápido
width = 160
height = 128
batch_size = 16
input_shape = (width, height,3)

## Datos almacenados
path_train = './data/fruits/train_data/'
path_val = './data/fruits/test_data/'

# Carga los datos de entrenamiento y validación
train_gen = load_data(path_train, width, height , batch_size)
val_gen = load_data(path_val, width, height , batch_size)

print(f"Train samples found: {train_gen.samples}")
print(f"Validation samples found: {val_gen.samples}")

# Crea y entrena el modelo simple
model = create_resnet_model(input_shape)
trained_model = train_model(model, train_gen, val_gen, epochs=10)

Found 49012 images belonging to 15 classes.
Found 4989 images belonging to 15 classes.
Train samples found: 49012
Validation samples found: 4989
Epoch 1/10
3064/3064 - 1594s - 520ms/step - accuracy: 0.2970 - loss: 2.2804 - val_accuracy: 0.1499 - val_loss: 2.4954
Epoch 2/10
3064/3064 - 1574s - 514ms/step - accuracy: 0.3490 - loss: 2.0897 - val_accuracy: 0.1958 - val_loss: 2.3769
Epoch 3/10
3064/3064 - 1687s - 551ms/step - accuracy: 0.3791 - loss: 1.9872 - val_accuracy: 0.2173 - val_loss: 2.2967
Epoch 4/10
3064/3064 - 1657s - 541ms/step - accuracy: 0.3984 - loss: 1.9183 - val_accuracy: 0.2489 - val_loss: 2.2342
Epoch 5/10
